In [1]:
import numpy as np
from numpy import *

In [420]:
# Explanation of data
#
#    The death rate is to be represented as a function of other variables.
#
#    There are 60 rows of data. Each point has 15 features:
#
#      A1,  the average annual precipitation;
#      A2,  the average January temperature;
#      A3,  the average July temperature;
#      A4,  the size of the population older than 65;
#      A5,  the number of members per household;
#      A6,  the number of years of schooling for persons over 22;
#      A7,  the number of households with fully equipped kitchens;
#      A8,  the population per square mile; 
#      A9,  the size of the nonwhite population;
#      A10, the number of office workers;
#      A11, the number of families with an income less than $3000;
#      A12, the hydrocarbon pollution index;
#      A13, the nitric oxide pollution index;
#      A14, the sulfur dioxide pollution index;
#      A15, the degree of atmospheric moisture.
#
#    The output is:
#      Y,   the death rate.
#      A and y; last column is y

In [421]:
import pandas as pd
# Upload data.
df = pd.DataFrame(pd.read_csv('deathrate_instance_python.dat', header=None, sep="\s+"))
df.columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'Y']

In [422]:
# Define matrix X and vector Y.
Y = df['Y']
X = df.drop(columns=['Y'])

In [308]:
# Add a column of ones.
X = pd.concat([X, pd.DataFrame({'O': np.ones(len(X))})], axis=1) 

In [309]:
# Transform dataframe into matrix.
Ymat = Y.values
Xmat = X.values

In [423]:
# Optimal values by AMPL.
wopt = np.array([0.48824,-0.0463881,0.102978,-0.0377061,0.00490736,-0.0339172,-0.255786,0.00564845,0.649708,-0.12622,0.213407,-0.207831,0.109842,0.376641,0.00995978])
gammaopt = 895.141
# Define it as 1 vector (list):
wopt = [0.48824,-0.0463881,0.102978,-0.0377061,0.00490736,-0.0339172,-0.255786,0.00564845,0.649708,-0.12622,0.213407,-0.207831,0.109842,0.376641,0.00995978,895.141]

In [428]:
"""
MINOS 5.51: optimal solution found.
133 iterations, objective 61484.65464
Nonlin evals: obj = 361, grad = 360, constrs = 361, Jac = 360.

norm2_w = -10785.1
gamma = 895.141

w [*] :=
 1   0.48824
 2  -0.0463881
 3   0.102978
 4  -0.0377061
 5   0.00490736
 6  -0.0339172
 7  -0.255786
 8   0.00564845
 9   0.649708
10  -0.12622
11   0.213407
12  -0.207831
13   0.109842
14   0.376641
15   0.00995978

"""
print()

In [438]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

In [432]:
# Loss func loss(x)
def loss_func (x):
    """
    Objective function: loss / cost function.
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    The objective function loss(x) evaluated at x.
    
    """
    w = np.asarray(x).T
    
    cost = (1/2) * ((Xmat @ w) - Ymat).T @ ((Xmat @ w) - Ymat)
  
    return np.squeeze(cost)

In [431]:
loss_func(wopt)

61484.65071643571

In [435]:
def loss_grad (x): 
    """
    Gradient of the objective function (loss / cost function).
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    Array of the gradient of the objective function loss(x) 
    evaluated at x.
    
    """
    w = np.asarray(x).T
    
    return Xmat.T @ (Xmat @ w - Ymat)

In [436]:
loss_grad(wopt) # valor optim

array([-1.05322859e+04,  9.99806306e+02, -2.22295893e+03,  8.13127974e+02,
       -1.05927484e+02,  7.31350810e+02,  5.51550115e+03, -2.10784096e+02,
       -1.40146068e+04,  2.72154380e+03, -4.60357165e+03,  4.48211610e+03,
       -2.36983501e+03, -8.12547179e+03, -2.16150552e+02, -2.28754456e-02])

In [437]:
def loss_hess (x):
    """
    Hessian matrix of the objective function (loss / cost function).
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    Matrix of the Hessian of the objective function loss(x) 
    evaluated at x.
    
    """
    w = np.asarray(x).T
    
    return Xmat.T @ Xmat

In [407]:
#loss_hess(wopt)

In [448]:
def cons_func (x):
    """
    Constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The constraint evaluated at x.
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    return np.sum(w**2)

In [449]:
cons_func(wopt)

1.0000003642408803

In [451]:
def cons_jacobian (x):
    """
    Gradient of the constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The gradient of the constraint evaluated at x.
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    return (2*w).tolist()

In [452]:
#cons_jacobian(wopt)

In [455]:
def cons_hess (x, v):
    """
    Product of the hessian of the constraint by an arbitrary vector.
    It is needed for the minimize function.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    v vector of length = number of constraints.
    
    Returns
    -------
    The Hessian of the constraint evaluated at x by v[0].
    
    """
    
    x[-1] = 0
    w = np.asarray(x).T
    H = (np.eye(w.shape[0])*2)
    H[-1,-1] = 0
    return v[0] * H

In [456]:
#cons_hess (w, np.ones(16))

In [460]:
# Define hyperparamater t:
t = 0

## OPTIMIZATION ##
# Assign initial random weights:
w = np.squeeze(np.random.rand(1,16))

# Define non linear constraint:
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, t, jac = cons_jacobian, hess = cons_hess) 

# Solve:
sol = minimize(loss_func, wopt, method='trust-constr', jac=loss_grad, hess=loss_hess,
               constraints=[nonlinear_constraint], options={'verbose': 1})
# Output:
print(sol)

`xtol` termination condition is satisfied.
Number of iterations: 291, function evaluations: 399, CG iterations: 710, optimality: 1.43e+08, constraint violation: 1.34e-09, execution time: 0.39 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 710
      cg_stop_cond: 2
            constr: [array([1.33535664e-09])]
       constr_nfev: [399]
       constr_nhev: [113]
       constr_njev: [112]
    constr_penalty: 1.0
  constr_violation: 1.3353566423295997e-09
    execution_time: 0.38890790939331055
               fun: 172931.85233161284
              grad: array([-2.12676666e+06, -1.96168194e+06, -4.21354097e+06, -4.95432059e+05,
       -1.84273797e+05, -6.17491954e+05, -4.55678992e+06, -2.20087356e+08,
       -6.90927336e+05, -2.59447465e+06, -8.17161544e+05, -2.07551454e+06,
       -1.25661070e+06, -3.13236449e+06, -3.24474690e+06, -5.64164813e+04])
               jac: [array([[ 3.20680571e-06, -2.21396002e-05, -3.38094393e-06,
    

1